# Baseline with mse loss with wide cnn
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=6
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=6


Using TensorFlow backend.


In [2]:
window_size = 2001
process_all = False
sample_num = 1000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 2001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 2001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 2001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 6
hidden_kernel_size_1 = 1000
# parameters for second conv layer
hidden_filters_2 = 6
hidden_kernel_size_2 = 600
# parameters for thirs conv layer
hidden_filters_3 = 6
hidden_kernel_size_3 = 400
# parameters for the output layer
output_filters = 1
output_kernel_size = 5
# parameters for training
batch_size = 64
num_epochs = 300
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
# 
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1)(inputs)
x = Dropout(dropout_rate)(x)


x = Conv1D(
    filters=hidden_filters_2,
    kernel_size=hidden_kernel_size_2,
    padding='same',
    activation='relu',
    strides=1)(x)
x = Dropout(dropout_rate)(x)

x = Conv1D(
    filters=hidden_filters_3,
    kernel_size=hidden_kernel_size_3,
    padding='same',
    activation='relu',
    strides=1)(x)
x = Dropout(dropout_rate)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2001, 6)           30006     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2001, 6)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2001, 6)           21606     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2001, 6)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2001, 6)           14406     
_________________________________________________________________
dropout_3 (Dropout)          (None, 2001, 6)           0         
__________

In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [22]:
model_dir = os.path.join("models", "mse_wide")
log_dir = os.path.join("logs", "mse_wide")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "mse_wide")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [23]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [24]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [25]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=7,
                              min_lr=1e-5)

In [26]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [27]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(7500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:7500], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(2500, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[7500:10000], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(1000, 2001)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[10000:11000], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [28]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [29]:
print "Fitting the model..."
X_train, y_train = atac_seq_day0[:7500], histone_mark_day0[:7500]
X_val, y_val = atac_seq_day0[7500:10000], histone_mark_day0[7500:10000]
X_test, y_test = atac_seq_day0[10000:11000], histone_mark_day0[10000:11000]

model.fit(atac_seq_day0[:10000],
          histone_mark_day0[:10000],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.25,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 7500 samples, validate on 2500 samples
Epoch 1/300
7488/7500 [============================>.] - ETA: 0s - loss: 434.9653 - pearson: 0.1034 - mean_squared_error: 434.9653 - mean_absolute_error: 7.91483- ETA: 18s - loss: 464.7164 - pearson: 0.0400 - mean_squared_error: 464.7164 - mean_ab - ETA: 11s - loss: 476.5933 - pearson: 0.0783 - mean_squared_error: 476.5933 - mean_absolute_error: - ETA: 9s - loss: 475.7688 - pearson: 0.0791 - mean_squared_error: 4 - ETA: 6s - loss: 449 - ETA: 1s - loss: 443.0711 - pearson: 0.1021 - mean_squared_error: 443.0711 - mean_abso
Epoch 00001: val_loss improved from inf to 168.43919, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Wr

Epoch 12/300
7488/7500 [============================>.] - ETA: 0s - loss: 388.5070 - pearson: 0.1790 - mean_squared_error: 388.5070 - mean_absolute_error: 8.2022
Epoch 00012: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 388.6932 - pearson: 0.1791 - mean_squared_error: 388.6932 - mean_absolute_error: 8.2057 - val_loss: 162.8450 - val_pearson: 0.1609 - val_mean_squared_error: 162.8450 - val_mean_absolute_error: 4.6205
Epoch 13/300
7488/7500 [============================>.] - ETA: 0s - loss: 395.5008 - pearson: 0.1753 - mean_squared_error: 395.5007 - mean_absolute_error: 8.1110
Epoch 00013: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 395.2508 - pearson: 0.1758 - mean_squared_error: 395.2508 - mean_absolute_error: 8.1110 - val_loss: 162.6591 - val_pearson: 0.1662 - val_mean_squared_error: 162.6591 - val_mean_absolute_error: 4.6220
Epoch 14/300
7488/7500 [============================>.] - ETA: 0s - 

Epoch 25/300
7488/7500 [============================>.] - ETA: 0s - loss: 376.8145 - pearson: 0.2028 - mean_squared_error: 376.8145 - mean_absolute_error: 8.1042
Epoch 00025: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 376.4979 - pearson: 0.2027 - mean_squared_error: 376.4979 - mean_absolute_error: 8.1015 - val_loss: 161.1008 - val_pearson: 0.1656 - val_mean_squared_error: 161.1008 - val_mean_absolute_error: 4.7498
Epoch 26/300
7488/7500 [============================>.] - ETA: 0s - loss: 375.0558 - pearson: 0.2032 - mean_squared_error: 375.0558 - mean_absolute_error: 8.0128
Epoch 00026: val_loss improved from 159.91113 to 159.61903, saving model to models/mse_wide/best_model.h5
Writing 1D track of shape: (7500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 2001)
Writing to file: /srv/www/kundaje/jesikmin/mse_wide/val.bw
Wrote bedgraph.

Epoch 39/300
7488/7500 [============================>.] - ETA: 0s - loss: 365.6423 - pearson: 0.2126 - mean_squared_error: 365.6423 - mean_absolute_error: 8.0126
Epoch 00039: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 365.9824 - pearson: 0.2133 - mean_squared_error: 365.9824 - mean_absolute_error: 8.0174 - val_loss: 161.4210 - val_pearson: 0.1685 - val_mean_squared_error: 161.4210 - val_mean_absolute_error: 4.6624
Epoch 40/300
1000/1000 [==============================] - 1s 708us/steps: 364.2804 - pearson: 0.2110 - mean_squared_error: 364.2804 - mean_
[('loss', 194.48692686462402), ('pearson', 0.26451178477704523), ('mean_squared_error', 194.4869256439209), ('mean_absolute_error', 5.6323980846405028)]
Test spearman: 0.268157738373

Epoch 00040: val_loss did not improve
7500/7500 [==============================] - 23s 3ms/step - loss: 364.0324 - pearson: 0.2117 - mean_squared_error: 364.0324 - mean_absolute_error: 8.0006 - val_loss: 160.18

Epoch 55/300
7488/7500 [============================>.] - ETA: 0s - loss: 351.8577 - pearson: 0.2302 - mean_squared_error: 351.8577 - mean_absolute_error: 7.8657
Epoch 00055: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 355.4783 - pearson: 0.2302 - mean_squared_error: 355.4783 - mean_absolute_error: 7.8861 - val_loss: 159.2873 - val_pearson: 0.1711 - val_mean_squared_error: 159.2873 - val_mean_absolute_error: 4.7739
Epoch 56/300
7488/7500 [============================>.] - ETA: 0s - loss: 355.8798 - pearson: 0.2310 - mean_squared_error: 355.8798 - mean_absolute_error: 7.9582
Epoch 00056: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 355.5067 - pearson: 0.2316 - mean_squared_error: 355.5067 - mean_absolute_error: 7.9557 - val_loss: 158.9386 - val_pearson: 0.1728 - val_mean_squared_error: 158.9386 - val_mean_absolute_error: 4.7831
Epoch 57/300
7488/7500 [============================>.] - ETA: 0s - 

...

Epoch 225/300
7488/7500 [============================>.] - ETA: 0s - loss: 345.0769 - pearson: 0.2554 - mean_squared_error: 345.0769 - mean_absolute_error: 7.8517
Epoch 00225: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 344.9861 - pearson: 0.2551 - mean_squared_error: 344.9861 - mean_absolute_error: 7.8527 - val_loss: 160.1143 - val_pearson: 0.1686 - val_mean_squared_error: 160.1143 - val_mean_absolute_error: 4.6283
Epoch 226/300
7488/7500 [============================>.] - ETA: 0s - loss: 345.8116 - pearson: 0.2470 - mean_squared_error: 345.8116 - mean_absolute_error: 7.8346
Epoch 00226: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 345.8040 - pearson: 0.2475 - mean_squared_error: 345.8040 - mean_absolute_error: 7.8319 - val_loss: 159.7606 - val_pearson: 0.1697 - val_mean_squared_error: 159.7606 - val_mean_absolute_error: 4.6408
Epoch 227/300
7488/7500 [============================>.] - ETA: 0s

Epoch 242/300
7488/7500 [============================>.] - ETA: 0s - loss: 343.8132 - pearson: 0.2488 - mean_squared_error: 343.8132 - mean_absolute_error: 7.8278
Epoch 00242: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 344.3387 - pearson: 0.2496 - mean_squared_error: 344.3387 - mean_absolute_error: 7.8310 - val_loss: 159.8409 - val_pearson: 0.1692 - val_mean_squared_error: 159.8409 - val_mean_absolute_error: 4.6490
Epoch 243/300
7488/7500 [============================>.] - ETA: 0s - loss: 345.3373 - pearson: 0.2568 - mean_squared_error: 345.3373 - mean_absolute_error: 7.8034
Epoch 00243: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 344.9572 - pearson: 0.2564 - mean_squared_error: 344.9572 - mean_absolute_error: 7.7990 - val_loss: 159.4399 - val_pearson: 0.1696 - val_mean_squared_error: 159.4399 - val_mean_absolute_error: 4.7152
Epoch 244/300
7488/7500 [============================>.] - ETA: 0s

7488/7500 [============================>.] - ETA: 0s - loss: 343.5730 - pearson: 0.2496 - mean_squared_error: 343.5730 - mean_absolute_error: 7.7511
Epoch 00259: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 343.1936 - pearson: 0.2497 - mean_squared_error: 343.1936 - mean_absolute_error: 7.7475 - val_loss: 159.8654 - val_pearson: 0.1686 - val_mean_squared_error: 159.8654 - val_mean_absolute_error: 4.6626
Epoch 260/300
1000/1000 [==============================] - 1s 700us/steps: 342.0738 - pearson: 0.2522 - mean_squared_error: 342.0738 - mean_
[('loss', 192.74304403686523), ('pearson', 0.24559628200531006), ('mean_squared_error', 192.74304663085937), ('mean_absolute_error', 5.5754465084075928)]
Test spearman: 0.262847967839

Epoch 00260: val_loss did not improve
7500/7500 [==============================] - 23s 3ms/step - loss: 342.7893 - pearson: 0.2524 - mean_squared_error: 342.7893 - mean_absolute_error: 7.8206 - val_loss: 159.8661 - val_pe

Epoch 276/300
7488/7500 [============================>.] - ETA: 0s - loss: 342.9400 - pearson: 0.2562 - mean_squared_error: 342.9400 - mean_absolute_error: 7.7731
Epoch 00276: val_loss did not improve
7500/7500 [==============================] - 20s 3ms/step - loss: 342.6669 - pearson: 0.2561 - mean_squared_error: 342.6669 - mean_absolute_error: 7.7731 - val_loss: 159.8589 - val_pearson: 0.1692 - val_mean_squared_error: 159.8589 - val_mean_absolute_error: 4.6485
Epoch 277/300
7488/7500 [============================>.] - ETA: 0s - loss: 342.4113 - pearson: 0.2594 - mean_squared_error: 342.4113 - mean_absolute_error: 7.7886
Epoch 00277: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 342.0461 - pearson: 0.2593 - mean_squared_error: 342.0461 - mean_absolute_error: 7.7846 - val_loss: 159.7626 - val_pearson: 0.1693 - val_mean_squared_error: 159.7626 - val_mean_absolute_error: 4.6629
Epoch 278/300
7488/7500 [============================>.] - ETA: 0s

Epoch 293/300
7488/7500 [============================>.] - ETA: 0s - loss: 340.3918 - pearson: 0.2607 - mean_squared_error: 340.3918 - mean_absolute_error: 7.7906
Epoch 00293: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 339.9775 - pearson: 0.2614 - mean_squared_error: 339.9775 - mean_absolute_error: 7.7851 - val_loss: 160.7374 - val_pearson: 0.1669 - val_mean_squared_error: 160.7374 - val_mean_absolute_error: 4.5829
Epoch 294/300
7488/7500 [============================>.] - ETA: 0s - loss: 340.4966 - pearson: 0.2577 - mean_squared_error: 340.4965 - mean_absolute_error: 7.7476
Epoch 00294: val_loss did not improve
7500/7500 [==============================] - 21s 3ms/step - loss: 340.2690 - pearson: 0.2578 - mean_squared_error: 340.2690 - mean_absolute_error: 7.7451 - val_loss: 160.3936 - val_pearson: 0.1679 - val_mean_squared_error: 160.3936 - val_mean_absolute_error: 4.5964
Epoch 295/300
7488/7500 [============================>.] - ETA: 0s